In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, Aer, assemble
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info.operators import Operator
import numpy as np
svsim = Aer.get_backend('aer_simulator')
simulator = QasmSimulator()

def dj_oracle(case, n):
    oracle_qc = QuantumCircuit(n+1)
    if case == "balanced":
        b = np.random.randint(1,2**n)
        b_str = format(b, '0'+str(n)+'b')
        for qubit in range(len(b_str)):
            if b_str[qubit] == '1':
                oracle_qc.x(qubit)
        for qubit in range(n):
            oracle_qc.cx(qubit, n)
        for qubit in range(len(b_str)):
            if b_str[qubit] == '1':
                oracle_qc.x(qubit)
    if case == "constant":
        output = np.random.randint(2)
        if output == 1:
            oracle_qc.x(n)
    oracle_gate = oracle_qc.to_gate()
    oracle_gate.name = "Oracle" 
    return oracle_gate

In [2]:
def dj_algorithm(oracle, n):
    dj_circuit = QuantumCircuit(n+1, n)
    dj_circuit.x(n)
    dj_circuit.h(n)
    for qubit in range(n):
        dj_circuit.h(qubit)
    dj_circuit.append(oracle, range(n+1))
    for qubit in range(n):
        dj_circuit.h(qubit)
    
    for i in range(n):
        dj_circuit.measure(i, i)
    
    return dj_circuit

In [3]:
oracle=dj_oracle("Balance",4)

In [4]:
a=dj_algorithm(oracle,4)
print(a.draw())

     ┌───┐     ┌─────────┐┌───┐┌─┐         
q_0: ┤ H ├─────┤0        ├┤ H ├┤M├─────────
     ├───┤     │         │├───┤└╥┘┌─┐      
q_1: ┤ H ├─────┤1        ├┤ H ├─╫─┤M├──────
     ├───┤     │         │├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─────┤2 Oracle ├┤ H ├─╫──╫─┤M├───
     ├───┤     │         │├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─────┤3        ├┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐│         │└───┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├┤4        ├──────╫──╫──╫──╫─
     └───┘└───┘└─────────┘      ║  ║  ║  ║ 
c: 4/═══════════════════════════╩══╩══╩══╩═
                                0  1  2  3 


In [5]:
compiled_circuit_a= transpile(a, simulator)
job_a = simulator.run(compiled_circuit_a, shots=1)
result_a = job_a.result()
counts_a = result_a.get_counts(compiled_circuit_a)
print("ccx:",counts_a)

ccx: {'0000': 1}


In [18]:
def DJ(n:int):
    oracleType, oracleValue = np.random.randint(2), np.random.randint(2)
    
    a = np.random.randint(1,2**n) # this is a hidden parameter for balanced oracle. 
    
    # Creating registers
    # n qubits for querying the oracle and one qubit for storing the answer
    qr = QuantumRegister(n+1) #all qubits are initialized to zero
    # for recording the measurement on the first register
    cr = ClassicalRegister(n)
    
    circuitName = "DeutschJozsa"
    djCircuit = QuantumCircuit(qr, cr)
    
    # Create the superposition of all input queries in the first register by applying the Hadamard gate to each qubit.
    for i in range(n):
        djCircuit.h(qr[i])

    # Flip the second register and apply the Hadamard gate.
    djCircuit.x(qr[n])
    djCircuit.h(qr[n])
        
    # Apply barrier to mark the beginning of the oracle
    djCircuit.barrier()
    
    for i in range(n):
        if (a & (1 << i)):
            djCircuit.cx(qr[i], qr[n])
            
    # Apply barrier to mark the end of the oracle
    djCircuit.barrier()
    
    # Apply Hadamard gates after querying the oracle
    for i in range(n):
        djCircuit.h(qr[i])
        
    # Measurement
    djCircuit.barrier()
    for i in range(n):
        djCircuit.measure(qr[i], cr[i])

    return (djCircuit,a)

In [19]:
programs=[DJ(5) for _ in range(20)]

In [21]:
results = [simulator.run(x[0], svsim, shots=128).result().get_counts() for x in programs]

In [22]:
results

[{'01001': 128},
 {'01011': 128},
 {'11001': 128},
 {'00110': 128},
 {'10000': 128},
 {'00101': 128},
 {'01110': 128},
 {'11101': 128},
 {'01100': 128},
 {'00100': 128},
 {'10100': 128},
 {'00111': 128},
 {'00110': 128},
 {'01011': 128},
 {'11111': 128},
 {'01011': 128},
 {'01000': 128},
 {'10011': 128},
 {'10000': 128},
 {'11011': 128}]

In [23]:
list(map(lambda x: x[1],programs))

[9, 11, 25, 6, 16, 5, 14, 29, 12, 4, 20, 7, 6, 11, 31, 11, 8, 19, 16, 27]

In [29]:
a = np.random.randint(1,32)
a

14

In [30]:
for i in range(5):
    print("i: ",i)
    print("1<<i: ",1 << i)
    print("a & (1 << i):  ",a & (1 << i))
    print("If a & (1 << i):  ",bool(a & (1 << i)))

i:  0
1<<i:  1
a & (1 << i):   0
If a & (1 << i):   False
i:  1
1<<i:  2
a & (1 << i):   2
If a & (1 << i):   True
i:  2
1<<i:  4
a & (1 << i):   4
If a & (1 << i):   True
i:  3
1<<i:  8
a & (1 << i):   8
If a & (1 << i):   True
i:  4
1<<i:  16
a & (1 << i):   0
If a & (1 << i):   False
